In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [30]:
#get the data from the csv file
df=pd.read_csv('bnpl_cleaned.csv')
print("Standard deviation of each feature:", np.std(df, axis=0))

Standard deviation of each feature: failed_traditional_credit          0.452831
bnpl_usage_frequency               4.391600
over_indebtedness_flag             0.480865
financial_stress_score             2.816011
external_repayment_loans           0.490302
credit_card_interest_incidence     0.475041
credit_limit_utilisation          26.250703
payment_delinquency_count          1.687891
impulsive_buying_score             2.903592
financial_literacy_assessment      2.973109
debt_accumulation_metric           1.452383
return_dispute_incidents           1.141574
demographic_risk_factor            1.433145
bnpl_debt_ratio                    0.570702
default_flag                       0.278882
dtype: float64


In [27]:
#Identify numeric columns
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Separate binary and continuous columns
binary_columns = [col for col in numeric_columns if df[col].nunique() == 2]
continuous_columns = [col for col in numeric_columns if df[col].nunique() > 2]

# Step 4: Print the results
print("Numeric columns:", numeric_columns)
print("Binary columns:", binary_columns)
print("Continuous columns (excluding binary):", continuous_columns)

Numeric columns: ['failed_traditional_credit', 'bnpl_usage_frequency', 'over_indebtedness_flag', 'financial_stress_score', 'external_repayment_loans', 'credit_card_interest_incidence', 'credit_limit_utilisation', 'payment_delinquency_count', 'impulsive_buying_score', 'financial_literacy_assessment', 'debt_accumulation_metric', 'return_dispute_incidents', 'demographic_risk_factor', 'bnpl_debt_ratio', 'default_flag']
Binary columns: ['failed_traditional_credit', 'over_indebtedness_flag', 'external_repayment_loans', 'credit_card_interest_incidence', 'default_flag']
Continuous columns (excluding binary): ['bnpl_usage_frequency', 'financial_stress_score', 'credit_limit_utilisation', 'payment_delinquency_count', 'impulsive_buying_score', 'financial_literacy_assessment', 'debt_accumulation_metric', 'return_dispute_incidents', 'demographic_risk_factor', 'bnpl_debt_ratio']


In [32]:
# Step 1: Initialize the scaler
scaler = StandardScaler()

# Step 2: Standardize the continuous columns
df[continuous_columns] = scaler.fit_transform(df[continuous_columns])

# Step 3: Verify the standardization
print(df[continuous_columns].head())
#print the mean

import numpy as np

#print the mean
print("Mean of each feature:", np.mean(df, axis=0))
print("Standard deviation of each feature:", np.std(df, axis=0))


   bnpl_usage_frequency  financial_stress_score  credit_limit_utilisation  \
0              1.546817                0.884585                 -0.299154   
1              0.863694                1.594809                 -1.632452   
2              1.319109               -0.535864                 -0.642002   
3             -0.047135               -1.246089                 -0.642002   
4             -1.185673               -1.246089                 -0.946756   

   payment_delinquency_count  impulsive_buying_score  \
0                  -0.315186                1.193694   
1                   1.462180                0.160491   
2                   1.462180               -0.872712   
3                   0.869724                0.160491   
4                  -0.315186                1.538095   

   financial_literacy_assessment  debt_accumulation_metric  \
0                       0.817999                 -0.656225   
1                      -1.536439                 -0.249996   
2             

In [34]:
#save the standardized dataframe to a new csv file
df.to_csv('bnpl_standardized.csv', index=False)